1. El Problema: Predicción de Precios de Casas
Vamos a simular un problema clásico de regresión. Tenemos un DataFrame con 3 columnas (Características o Features):

* Metros Cuadrados (ej: 50 a 200).

* Antigüedad (ej: 1 a 50 años).

* Distancia al centro (ej: 0.5 a 20 km).

Y queremos predecir el Precio (Target).

2. El Puente: De Pandas a PyTorch
Aquí hay una regla de oro que un Senior Engineer nunca olvida:

⚠️ Regla de Oro: A las redes neuronales NO les gustan los números grandes ni los rangos dispares.

Si metes "100 metros" y "0.5 km" juntos, la red se confundirá porque 100 es mucho más grande que 0.5, aunque la distancia sea importante. Solución: Debemos normalizar los datos (hacer que todo esté más o menos entre -1 y 1) antes de convertirlos a Tensores.

Usaremos StandardScaler de Scikit-Learn para esto.

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 1. Simulación de un CSV cargado en Pandas
data = {
    'metros': np.random.randint(50, 200, 1000),
    'antiguedad': np.random.randint(1, 50, 1000),
    'distancia': np.random.uniform(0.5, 20, 1000),
    'precio': np.zeros(1000) # Placeholder
}
df = pd.DataFrame(data)

# Generamos un precio lógico: Precio base + (Metros * 1000) - (Antigüedad * 500) ...
df['precio'] = 50000 + (df['metros'] * 1000) - (df['antiguedad'] * 500) - (df['distancia'] * 1000)

# 2. Separamos Features (X) y Target (y)
X_numpy = df[['metros', 'antiguedad', 'distancia']].values
y_numpy = df['precio'].values.reshape(-1, 1) # Importante: reshape a columna

# 3. NORMALIZACIÓN (CRÍTICO PARA REDES NEURONALES)
scaler_x = StandardScaler()
scaler_y = StandardScaler() # Escalamos también el precio para que la Loss no sea gigante

X_scaled = scaler_x.fit_transform(X_numpy)
y_scaled = scaler_y.fit_transform(y_numpy)

print(f"Datos listos.")
print(f"Ejemplo X normalizado: {X_scaled[0]}")
print(f"Ejemplo y normalizado: {y_scaled[0]}")

Datos listos.
Ejemplo X normalizado: [ 0.16292116 -0.20180036  0.22807746]
Ejemplo y normalizado: [0.16163998]


Tu Misión:

1) Crea la clase HouseDataset:

* En el __init__, debe recibir dos argumentos: features y targets.

* Dentro del __init__, convierte esos arrays de numpy a Tensores de PyTorch tipo float32.

* Pista: torch.tensor(..., dtype=torch.float32)

Implementa __len__ y __getitem__.

2) Crea la clase HouseModel:

* Entrada: 3 neuronas (porque tenemos 3 features).

* Oculta: 2 capas ocultas de 64 neuronas cada una con ReLU.

* Salida: 1 neurona (el precio). ¡OJO! No uses activación en la salida final (queremos un valor lineal continuo, no una probabilidad).

Escribe el código de estas dos clases y instáncialas.

Instancia el dataset pasando X_scaled y y_scaled.

Instancia el modelo.

In [5]:
class HouseDataset(Dataset):
    def __init__(self, features, targets):
        self.features=torch.tensor(data=features, dtype=torch.float32)
        self.targets=torch.tensor(data=targets, dtype=torch.float32)
    
    def __len__(self) -> int:
        return len(self.features)
    
    def __getitem__(self, index):
        # DEVOLVEMOS TUPLA: (Datos, Precio)
        return self.features[index], self.targets[index]

class HouseModel(nn.Module):
    def __init__(self):
        super().__init__()
        #primera de 3 a 64 ocultas. Entrada -> Oculta
        self.capa1=nn.Linear(in_features=3,out_features=64)
        self.activacion=nn.ReLU()
        #oculta -> oculta. segunda transformación (Profundidad): De 64 a 64
        self.capa2=nn.Linear(in_features=64,out_features=64)
        #capa Final (Compresión): De 64 a 1 solo precio
        self.salida=nn.Linear(in_features=64,out_features=1)
    
    def forward(self, x):
        """Debe pasar x a través de: capa1 -> activacion -> capa2 -> activacion -> salida."""
        x=self.capa1(x)
        x=self.activacion(x)
        x=self.capa2(x)
        x=self.activacion(x)
        x=self.salida(x)
        return x

modelo=HouseModel()
dataset=HouseDataset(features=X_scaled, targets=y_scaled)

Vas a escribir el script de entrenamiento completo tú solo, uniendo todo lo que hemos visto.

Requisitos Técnicos:

* DataLoader: Instáncialo con tu dataset, batch_size=32 y shuffle=True.

* Loss Function: Como es un problema de regresión (predecir valores continuos), DEBES usar nn.MSELoss() (Error Cuadrático Medio).

* Optimizador: Usa torch.optim.Adam con un learning rate (lr) de 0.001. Adam suele funcionar mejor que SGD para estos datos.

* Bucle: Entrena por 20 épocas.

In [7]:
#Crear el DataLoader
#batch_size=32: Entregará paquetes de 32 datos
#shuffle=True: Barajará los datos en cada época
loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

#loss function
loss_function=nn.MSELoss()
#optimizador
optimizador=torch.optim.Adam(params=modelo.parameters(),lr=0.001)
#bucle
epochs=20

for epoch in range(epochs):
    loss_acum = 0
    
    for batch_x, batch_y in loader:      
        # 1. Forward
        prediccion = modelo(batch_x)
        
        # 2. Calcular Loss
        loss = loss_function(prediccion, batch_y)
        
        # 3. Limpiar gradientes previos
        optimizador.zero_grad()
        
        # 4. Backward (Calcular gradientes)
        loss.backward()
        
        # 5. Actualizar pesos
        optimizador.step()
        
        # Acumulamos el error para monitorear (opcional)
        loss_acum += loss.item()
    
    # Reporte por epoch
    promedio_loss = loss_acum / len(loader)
    print(f"Época {epoch+1} | Loss Promedio: {promedio_loss:.5f}")

Época 1 | Loss Promedio: 0.00039
Época 2 | Loss Promedio: 0.00020
Época 3 | Loss Promedio: 0.00014
Época 4 | Loss Promedio: 0.00014
Época 5 | Loss Promedio: 0.00012
Época 6 | Loss Promedio: 0.00014
Época 7 | Loss Promedio: 0.00013
Época 8 | Loss Promedio: 0.00017
Época 9 | Loss Promedio: 0.00013
Época 10 | Loss Promedio: 0.00009
Época 11 | Loss Promedio: 0.00007
Época 12 | Loss Promedio: 0.00011
Época 13 | Loss Promedio: 0.00017
Época 14 | Loss Promedio: 0.00008
Época 15 | Loss Promedio: 0.00011
Época 16 | Loss Promedio: 0.00014
Época 17 | Loss Promedio: 0.00007
Época 18 | Loss Promedio: 0.00010
Época 19 | Loss Promedio: 0.00005
Época 20 | Loss Promedio: 0.00005


Llega un cliente a tu inmobiliaria.

La Casa:Metros: 120 $m^2$

Antigüedad: 10 años

Distancia: 5.0 km

Tu Misión:Usa tu modelo entrenado para predecir el precio de esta casa.

In [8]:
# 1. Definimos los datos de la casa nueva (en el mismo orden que entrenamos)
# [metros, antiguedad, distancia]
casa_nueva = np.array([[120, 10, 5.0]]) 

# 2. Normalizamos la entrada (Usamos el MISMO scaler_x del entrenamiento)
casa_nueva_scaled = scaler_x.transform(casa_nueva)

# 3. Convertimos a Tensor (recuerda el tipo de dato y el dispositivo)
# RELLENA AQUI: casa_tensor = ...
casa_tensor = torch.tensor(casa_nueva_scaled, dtype=torch.float32)

# 4. Predicción
modelo.eval() # Modo evaluación
with torch.no_grad(): # Apagamos gradientes
    # RELLENA AQUI: Haz el pase forward
    prediccion_scaled = modelo(casa_tensor)

# 5. Des-normalizamos el precio (De vuelta a dólares)
# Usamos scaler_y.inverse_transform
# OJO: inverse_transform espera un array de numpy, no un tensor.
precio_final = scaler_y.inverse_transform(prediccion_scaled.numpy())

print(f"El precio estimado es: ${precio_final[0][0]:,.2f}")

El precio estimado es: $160,196.98
